# Imports

##### General imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm
import pprint

##### Import from flatland environment 

In [3]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator,sparse_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator, sparse_schedule_generator
from flatland.utils.rendertools import RenderTool, AgentRenderVariant

##### Import from our framework

In [4]:
from src.graph import NetworkGraph

In [5]:
from src.flows import *

# Test of time expanded network

##### Create a flatland network

sparse netwrok

In [ ]:
if False:
    number_agents = 5

    size_side = 50
    stochastic_data = {'prop_malfunction': 0.3,  # Percentage of defective agents
                       'malfunction_rate': 30,  # Rate of malfunction occurence
                       'min_duration': 3,  # Minimal duration of malfunction
                       'max_duration': 20  # Max duration of malfunction
                       }
    speed_ration_map = {1.: 0.25,  # Fast passenger train
                        1. / 2.: 0.25,  # Fast freight train
                        1. / 3.: 0.25,  # Slow commuter train
                        1. / 4.: 0.25}  # Slow freight train
    env = RailEnv(width=size_side,
                  height=size_side,
                  rail_generator=sparse_rail_generator(max_num_cities=4,
                                                       # Number of cities in map (where train stations are)
                                                       seed=14,  # Random seed
                                                       grid_mode=False,
                                                       max_rails_between_cities=2,
                                                       max_rails_in_city=8,
                                                       ),
                  schedule_generator=sparse_schedule_generator(speed_ration_map),
                  number_of_agents=number_agents,
                  stochastic_data=stochastic_data,  # Malfunction data generator
                  obs_builder_object=GlobalObsForRailEnv(),
                  remove_agents_at_target=True
                  )

    # RailEnv.DEPOT_POSITION = lambda agent, agent_handle : (agent_handle % env.height,0)
    env.reset()

    env_renderer = RenderTool(env,
                              agent_render_variant=AgentRenderVariant.AGENT_SHOWS_OPTIONS_AND_BOX,
                              show_debug=True,
                              screen_height=1100,
                              screen_width=1800)
    env_renderer.reset()

    env_renderer.render_env(show=True, show_observations=False, show_predictions=False)



usual network

In [6]:
number_agents = 50

size_side = 50
env = RailEnv(width=size_side,
              height=size_side,
              rail_generator=complex_rail_generator(nr_start_goal=number_agents, nr_extra=1, 
                                                    min_dist=6, max_dist=99999, 
                                                    seed = np.random.randint(0,2000)),
              schedule_generator=complex_schedule_generator(),
              number_of_agents=number_agents,
              obs_builder_object=GlobalObsForRailEnv())

env.reset()



env_renderer = RenderTool(env)
env_renderer.render_env(show=True, show_predictions=False, show_observations=False)

In [7]:
matrix_rail = np.array(env.rail.grid.tolist())
flatlandNetwork = NetworkGraph(matrix_rail,[(0,1)],[(1,0)])

In [8]:
sources = []
sinks = []
directions = []
for agent in env.agents:
    sources.append(agent.initial_position)
    sinks.append(agent.target)
    try:
        directions.append(agent.direction)
    except:
        pass

In [9]:
if len(directions) == 0:
    directions = None

##### create a time expanded network

In [10]:
import time

In [11]:
start = time.time()
TestNetworkTime = TimeNetwork(flatlandNetwork, depth=200)
stop = time.time()
print(f'time taken to build the graph: {stop-start}')

time taken to build the graph: 31.811854600906372


In [12]:
TestNetworkTime.connect_sources_and_sink(sources,sinks,directions = None)

### create initial solution

In [13]:
from tqdm import tqdm

In [14]:
constraints, find_constraints = TestNetworkTime.get_topology_network()

In [15]:
test = InitialSolutionGenerator(TestNetworkTime,constraints,find_constraints,len(sources))

In [16]:
solution = test.getInitialSolution()

50it [17:58, 21.57s/it] 


In [17]:
test.showStats()

{'not shortest path for 16': 14,
 'not shortest path for 19': 6,
 'not shortest path for 26': 2,
 'not shortest path for 27': 20,
 'not shortest path for 30': 16,
 'not shortest path for 37': 19,
 'not shortest path for 41': 67,
 'not shortest path for 42': 8,
 'not shortest path for 43': 26,
 'not shortest path for 44': 55,
 'not shortest path for 47': 35,
 'not shortest path for 48': 2,
 'not shortest path for 49': 22}


In [18]:
len(solution)

50

In [19]:
cost = 0
for p in solution:
    cost += len(p)
    
print(cost)

4158


In [20]:
from src.flows.PricingProblem import PricingSolver

In [21]:
start = time.time()
flag = True
iteration = 1
master = MasterProblem.MasterProblem(solution,constraints,find_constraints,number_agents)
master.build()
pricingSolver = PricingSolver(TestNetworkTime.graph,constraints,find_constraints,len(sources))
while flag:
    print("iteration ",iteration)
    master.solveRelaxedModel()
    duals = master.getDualVariables()
    pathsToAdd, flag = pricingSolver.get_columns_to_add(master.getDualVariables(),master.constraintsActivated)
    if flag:
        #pprint.pprint(pathsToAdd)
        print(f" commodities with updated paths: {pathsToAdd.keys()}")
        iteration+= 1
        master.addColumn(pathsToAdd)
print(f"column generation method took {np.round(time.time()-start,3)} sec")

Academic license - for non-commercial use only
iteration  1
Optimize a model with 6060 rows, 50 columns and 6060 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 50 rows and 6110 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.1580000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  4.158000000e+03
certificat for ('source_16', 'sink_16'): weight is 129.0, sigma is 143.0
certificat for ('source_19', 'sink_19'): weight is 49.0, sigma is 55.0
certificat for ('source_26', 'sink_26'): weight is 73.0, sigma is 75.0
certificat for ('source_27', 'sink_27'): weight is 97.0, sigma is 117.0
certificat for ('source_30', 'sink_30'): weight is 149.0, sigma is 165.0
certificat for ('source_37', 'sink_37'): weight is 1

 commodities with updated paths: dict_keys([0, 10, 16, 32, 42, 47])
iteration  6
Optimize a model with 12075 rows, 118 columns and 15911 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 50 rows and 11916 columns
Presolve time: 0.01s
Presolved: 68 rows, 277 columns, 663 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9520000e+03   0.000000e+00   1.600000e+01      0s
      28    3.9565000e+03   0.000000e+00   0.000000e+00      0s

Solved in 28 iterations and 0.02 seconds
Optimal objective  3.956500000e+03
certificat for ('source_7', 'sink_7'): weight is 80.0, sigma is 81.0
certificat for ('source_16', 'sink_16'): weight is 130.0, sigma is 143.0
certificat for ('source_21', 'sink_21'): weight is 86.0, sigma is 87.0
certificat for ('source_37', 'sink_37'): weight is 178.5, sigma is 179.5
certificat for ('source_42',

In [22]:
master.solveRelaxedModel()

Optimize a model with 12735 rows, 128 columns and 17386 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 41 rows and 12508 columns
Presolve time: 0.02s
Presolved: 87 rows, 355 columns, 870 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7620000e+03   0.000000e+00   1.800000e+01      0s
      33    3.9560000e+03   0.000000e+00   0.000000e+00      0s

Solved in 33 iterations and 0.03 seconds
Optimal objective  3.956000000e+03


In [23]:
master.relaxedModel.objVal

3956.0

In [24]:
master.model.optimize()

Optimize a model with 12735 rows, 128 columns and 17386 nonzeros
Variable types: 0 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4158.0000000
Presolve removed 12727 rows and 120 columns
Presolve time: 0.01s
Presolved: 8 rows, 8 columns, 21 nonzeros
Found heuristic solution: objective 4067.0000000
Variable types: 0 continuous, 8 integer (8 binary)

Root relaxation: cutoff, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      4067.00000 4067.00000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 4067 4073 

Optimal solution found (tolerance 1.00e-04)
Best 

In [26]:
for v in master.relaxedModel.getVars():
    print('%s %g' % (v.varName, v.x))

path[0,0] 0.5
path[1,0] 0.5
path[2,0] 1
path[3,0] 1
path[4,0] 0.5
path[5,0] 1
path[6,0] 0
path[7,0] 0.5
path[8,0] 1
path[9,0] 1
path[10,0] 1
path[11,0] 0.5
path[12,0] 0.5
path[13,0] 1
path[14,0] 1
path[15,0] 1
path[16,0] 0
path[17,0] 1
path[18,0] 1
path[19,0] 0
path[20,0] 1
path[21,0] 0.5
path[22,0] 1
path[23,0] 1
path[24,0] 1
path[25,0] 1
path[26,0] 0.5
path[27,0] 0
path[28,0] 1
path[29,0] 1
path[30,0] 0
path[31,0] 1
path[32,0] 0
path[33,0] 1
path[34,0] 1
path[35,0] 1
path[36,0] 1
path[37,0] 0
path[38,0] 1
path[39,0] 1
path[40,0] 1
path[41,0] 0
path[42,0] 0
path[43,0] 0
path[44,0] 0
path[45,0] 1
path[46,0] 1
path[47,0] 0
path[48,0] 0
path[49,0] 0
path[16,1] 0
path[19,1] 0.5
path[26,1] 0.5
path[27,1] 0.5
path[30,1] 0
path[37,1] 0
path[41,1] 0
path[42,1] 0.5
path[43,1] 0.5
path[44,1] 0
path[47,1] 0.5
path[48,1] 1
path[49,1] 0.5
path[0,1] 0
path[1,1] 0
path[2,1] 0
path[4,1] 0
path[6,1] 1
path[8,1] 0
path[11,1] 0
path[12,1] 0
path[16,2] 0
path[19,2] 0
path[21,1] 0.5
path[26,2] 0
path[27,2

In [25]:
for v in master.model.getVars():
    print('%s %g' % (v.varName, v.x))

path[0,0] 1
path[1,0] 1
path[2,0] 1
path[3,0] 1
path[4,0] 1
path[5,0] 1
path[6,0] -0
path[7,0] 1
path[8,0] 1
path[9,0] 1
path[10,0] 1
path[11,0] 1
path[12,0] 1
path[13,0] 1
path[14,0] 1
path[15,0] 1
path[16,0] 0
path[17,0] 1
path[18,0] 1
path[19,0] 1
path[20,0] 1
path[21,0] 1
path[22,0] 1
path[23,0] 1
path[24,0] 1
path[25,0] 1
path[26,0] 1
path[27,0] 1
path[28,0] 1
path[29,0] 1
path[30,0] 0
path[31,0] 1
path[32,0] 0
path[33,0] 1
path[34,0] 1
path[35,0] 1
path[36,0] 1
path[37,0] 1
path[38,0] 1
path[39,0] 1
path[40,0] 1
path[41,0] 0
path[42,0] 0
path[43,0] 1
path[44,0] 0
path[45,0] 1
path[46,0] 1
path[47,0] 1
path[48,0] 1
path[49,0] 1
path[16,1] 0
path[19,1] 0
path[26,1] 0
path[27,1] 0
path[30,1] 1
path[37,1] 0
path[41,1] 0
path[42,1] -0
path[43,1] 0
path[44,1] 0
path[47,1] 0
path[48,1] 0
path[49,1] 0
path[0,1] 0
path[1,1] 0
path[2,1] 0
path[4,1] 0
path[6,1] 1
path[8,1] 0
path[11,1] 0
path[12,1] 0
path[16,2] 1
path[19,2] 0
path[21,1] 0
path[26,2] 0
path[27,2] 0
path[30,2] 0
path[32,1] 1


In [27]:
master.cost

{(0, 0): 63,
 (1, 0): 69,
 (2, 0): 63,
 (3, 0): 79,
 (4, 0): 67,
 (5, 0): 65,
 (6, 0): 97,
 (7, 0): 79,
 (8, 0): 89,
 (9, 0): 93,
 (10, 0): 109,
 (11, 0): 55,
 (12, 0): 93,
 (13, 0): 79,
 (14, 0): 69,
 (15, 0): 71,
 (16, 0): 143,
 (17, 0): 99,
 (18, 0): 47,
 (19, 0): 55,
 (20, 0): 45,
 (21, 0): 85,
 (22, 0): 59,
 (23, 0): 51,
 (24, 0): 67,
 (25, 0): 87,
 (26, 0): 75,
 (27, 0): 117,
 (28, 0): 129,
 (29, 0): 21,
 (30, 0): 165,
 (31, 0): 73,
 (32, 0): 115,
 (33, 0): 61,
 (34, 0): 51,
 (35, 0): 33,
 (36, 0): 57,
 (37, 0): 187,
 (38, 0): 37,
 (39, 0): 117,
 (40, 0): 51,
 (41, 0): 159,
 (42, 0): 51,
 (43, 0): 165,
 (44, 0): 174,
 (45, 0): 33,
 (46, 0): 43,
 (47, 0): 140,
 (48, 0): 47,
 (49, 0): 79,
 (16, 1): 129,
 (19, 1): 49,
 (26, 1): 73,
 (27, 1): 97,
 (30, 1): 149,
 (37, 1): 177,
 (41, 1): 131,
 (42, 1): 43,
 (43, 1): 139,
 (44, 1): 147,
 (47, 1): 105,
 (48, 1): 45,
 (49, 1): 57,
 (0, 1): 65,
 (1, 1): 70,
 (2, 1): 64,
 (4, 1): 68,
 (6, 1): 97,
 (8, 1): 90,
 (11, 1): 56,
 (12, 1): 94,
 (1

In [28]:
master.model.write("temp/test.lp")

##### Test LP Formulation

test a simple graph

In [ ]:
import datetime as dt
print(dt.datetime.now())

In [ ]:
constraints,findConstraints = TestNetworkTime.get_topology_network()

In [ ]:
mcflow = MCFlow(TestNetworkTime.graph,len(sources),constraints,integer = True)

In [ ]:
mcflow.solve()

In [ ]:
paths = mcflow.solution_complete_edges

In [ ]:
score = 0
for _,path in paths.items():
    score += len(path)
    
print(f"score {score}")

In [ ]:
paths

In [ ]:
lengths = []
pathsToAllongate = copy.deepcopy(mcflow.solution)
for agent,path in pathsToAllongate.items():
    lengths.append(len(path))
    
maxLength = max(lengths)
for agent,path in pathsToAllongate.items():
    for i in range(maxLength-len(path)):
        path.append(None)
    pathsToAllongate[agent] = path

In [ ]:
dfPaths = pd.DataFrame(pathsToAllongate)
dfPaths

In [ ]:
import datetime as dt
print(dt.datetime.now())

In [ ]:
colors = [(230, 25, 75), (60, 180, 75), (255, 225, 25), (0, 130, 200), (245, 130, 48),
          (145, 30, 180), (70, 240, 240), (240, 50, 230), (210, 245, 60), (250, 190, 190),
          (0, 128, 128), (230, 190, 255), (170, 110, 40), (255, 250, 200), (128, 0, 0), (170, 255, 195),
          (128, 128, 0), (255, 215, 180), (0, 0, 128), (128, 128, 128), (255, 255, 255), (0, 0, 0)]
colors = [(x[0]/255.,x[1]/255.,x[2]/255.) for x in colors]

In [ ]:
flatlandNetwork.show(paths = mcflow.solution_complete_edges)